# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
outputfile = "../output_data/cities.csv"
weather_data_df = pd.read_csv(outputfile)
weather_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Yellowknife,62.46,-114.35,21.20,92,90,16.11,CA,1604178627
1,1,Bethel,41.37,-73.41,45.00,39,1,6.93,US,1604178781
2,2,Vaini,-21.20,-175.20,75.20,83,75,11.41,TO,1604178782
3,3,George Town,5.41,100.34,75.20,94,20,4.70,MY,1604178782
4,4,Alta Floresta,-9.88,-56.09,85.71,58,100,2.33,BR,1604178782
...,...,...,...,...,...,...,...,...,...,...
544,544,Pochutla,15.74,-96.47,88.00,62,20,9.17,MX,1604179224
545,545,Achacachi,-16.05,-68.68,51.82,63,22,11.97,BO,1604179224
546,546,Ortigueira,-24.21,-50.95,66.60,69,30,5.66,BR,1604179224
547,547,Ardakān,32.31,54.02,57.20,22,0,2.30,IR,1604179224


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [20]:
locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df['Humidity']
figure_layout = {
    'width': '400px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '3px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
max_humidity = max(humidity)
heat_layer.max_intensity = max_humidity
heat_layer.point_radius = 2
#heat_layer.gradient = ['white', 'red']

The picture is not as good as yours but any and all amount of tweaking got me to this.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
perfect_weather = weather_data_df.loc[(weather_data_df['Max Temp'] > 70) & (weather_data_df['Max Temp']  <= 82) & 
                                      (weather_data_df['Cloudiness'] <5) & 
                                      (weather_data_df['Wind Speed'] <5) ]

perfect_weather.drop(perfect_weather.columns[1], axis=1)
perfect_weather = perfect_weather.drop(perfect_weather.columns[0], axis=1)
perfect_weather = perfect_weather.reset_index()
perfect_weather = perfect_weather.dropna()
perfect_weather

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,10,Bonthe,7.53,-12.51,79.25,80,0,4.29,SL,1604178782
1,25,Marrakesh,31.63,-8.01,75.20,36,0,4.70,MA,1604178488
2,45,Saint George,37.10,-113.58,75.00,15,1,0.74,US,1604178784
3,135,Tiznit Province,29.58,-9.50,71.89,25,0,4.52,MA,1604178871
4,155,Salalah,17.02,54.09,75.20,41,0,4.70,OM,1604178913
5,191,Bosaso,11.28,49.18,77.63,80,0,1.97,SO,1604178916
6,212,Laguna,38.42,-121.42,79.00,22,1,3.36,US,1604178958
7,232,Uthal,25.81,66.62,76.08,30,0,3.40,PK,1604178959
8,236,Galveston,29.37,-94.97,71.60,53,1,4.70,US,1604178920
9,259,Ampanihy,-24.70,44.75,71.73,61,0,4.99,MG,1604179001


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
hotel_df = pd.DataFrame({'Hotel Name': "",
                        'City': perfect_weather['City'],
                        'Country': perfect_weather['Country'],
                         'Lat':perfect_weather['Lat'],
                         'Lng':perfect_weather['Lng']
                        }
                       )
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,,Bonthe,SL,7.53,-12.51
1,,Marrakesh,MA,31.63,-8.01
2,,Saint George,US,37.10,-113.58
3,,Tiznit Province,MA,29.58,-9.50
4,,Salalah,OM,17.02,54.09
5,,Bosaso,SO,11.28,49.18
6,,Laguna,US,38.42,-121.42
7,,Uthal,PK,25.81,66.62
8,,Galveston,US,29.37,-94.97
9,,Ampanihy,MG,-24.70,44.75


I understand that you have mentioned 10 or less rows. But, I had to take a larger set due to lack of hotel names in the area. Finally when I got one I decided to take a snapshot and keep it for your perusal. If my data does not match the picture it is beacuse I left a snapshot when I was able to get values. I hope that is ok.

In [27]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "keyword": "motel",
    "key": g_key
}
# Use the lat/lng we recovered to identify cities with perfect weather
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

     # add location each iteration 
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "4 star" and our lat/lng ( I ran a search with lat amd lng the results were bad)
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#   print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [28]:
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,,Bonthe,SL,7.53,-12.51
1,La Mamounia,Marrakesh,MA,31.63,-8.01
2,Dixie Palms Motel St. George,Saint George,US,37.10,-113.58
3,,Tiznit Province,MA,29.58,-9.50
4,Star Emirates Suites,Salalah,OM,17.02,54.09
5,Hotel Roble,Bosaso,SO,11.28,49.18
6,,Laguna,US,38.42,-121.42
7,"Girls Hostel, LUAWMS",Uthal,PK,25.81,66.62
8,"Motel 6 Texas City, TX - I-45 South",Galveston,US,29.37,-94.97
9,,Ampanihy,MG,-24.70,44.75


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

I had to take a snapshot of from my coumputer. Downloading does not give me the info popup.